<img width="8%" alt="Snowflake.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Snowflake.png" style="border-radius: 15%">

# Snowflake - Ingest csv data from local stage
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Snowflake+-+Ingest+csv+data+from+local+stage:+Error+short+description">Bug report</a>

**Tags:** #snowflake #data #warehouse #naas_drivers #snippet

**Author:** [Mateusz Polakowski](https://www.linkedin.com/in/polakowski/)

**Last update:** 2023-04-12 (Created: 2022-08-06)

**Description:** This notebook demonstrates how to ingest CSV data from a local stage into Snowflake.

## Input

### Import library

In [1]:
import os
from naas_drivers import snowflake
from snowflake.connector.errors import ProgrammingError

### Setup Snowflake account

If you don't have your SF account, you can easily set up a [30-day trial account with $400 budget here](https://signup.snowflake.com/).

To get your Snowflake account ID essential for connecting, please refer to [Account Identifiers in Snowflake documentation](https://docs.snowflake.com/en/user-guide/admin-account-identifier.html). There are several methods to get your account ID, but the overall rule can be found below:

```<account_identifier>.snowflakecomputing.com```

If you're proceeding with the trial account, it's highly probable that your ID will resemble something like: `xy1234.eu-central-1`.

### Credentials

In [2]:
# Here environment variables are used to pass Snowflake credentials,
# but it's okay to do it in a different manner

sf_username = os.environ["SNOWFLAKE_USER"]
sf_password = os.environ["SNOWFLAKE_PASSWORD"]
sf_account = os.environ["SNOWFLAKE_ACCOUNT"]

## Model

### Connecting to your Snowflake account

In [3]:
snowflake.connect(username=sf_username, password=sf_password, account=sf_account)

### Environment setup

In [4]:
snowflake.database.get_current() is None

In [5]:
snowflake.warehouse.use("COMPUTE_WH")

In [6]:
snowflake.role.use("ACCOUNTADMIN")

In [7]:
snowflake.database.create("NAAS", or_replace=True, silent=True)
snowflake.database.use("NAAS", silent=True)
snowflake.schema.create("NAAS_SCHEMA", or_replace=True, silent=True)
snowflake.schema.use("NAAS_SCHEMA", silent=True)

In [8]:
snowflake.get_environment()

## Output

### Creating file format

In [9]:
results_ff = snowflake.file_format.create(
    "my_csv_format", "CSV", or_replace=True, FIELD_DELIMITER="','", SKIP_HEADER=1
)

results_ff

### Creating file format with wrong parameter (not applicable for JSON file format, see [docs](https://docs.snowflake.com/en/sql-reference/sql/create-file-format.html))

In [10]:
try:
    snowflake.file_format.create(
        "my_invalid_csv_format", "CSV", or_replace=True, ENABLE_OCTAL="TRUE"
    )
except ProgrammingError as pe:
    print("Something went wrong!")
    print(pe)

### Dropping file format

In [11]:
# snowflake.file_format.drop('my_csv_format', if_exists=True)

### Creating internal stage

In [12]:
snowflake.stage.create(
    stage_name="my_internal_stage", or_replace=True, file_format_name="my_csv_format"
)

### Putting data on the stage

In [13]:
snowflake.stage.put(
    filepath="file://~/Downloads/sales_data.csv",
    internal_stage_name="@my_internal_stage",
    silent=True,
)

### Showing files inside stage

In [14]:
result_list_stage = snowflake.stage.list("@my_internal_stage")

print(result_list_stage["results"])

### Creating a table

It's possible to do something like `CREATE TABLE AS SELECT ... FROM @stage`, although it's highly not recommended! With that approach we don't keep file loading history (which Snowflake is capable to do and make a lot of use from).

In [15]:
query_create_table = """
    CREATE OR REPLACE TABLE sales_data (
      month VARCHAR
      , year INT
      , item VARCHAR
      , amount BIGINT
      , currency VARCHAR
    );
"""

# No worries, Table API will be available soon too!
snowflake.execute(query_create_table)["results"]

### Loading data from internal stage to a table

In [16]:
snowflake.copy_into(
    table_name="sales_data", source_stage="@my_internal_stage", silent=True
)

### Running the same command again 

No harm as `COPY INTO` tracks which files have been loaded already

In [17]:
snowflake.copy_into(
    table_name="sales_data", source_stage="@my_internal_stage", silent=True
)

### Data querying straight to Pandas DataFrame

In [18]:
sales_data = snowflake.query_pd("SELECT * FROM sales_data")
sales_data.head()